In [ ]:
# Colab cell 1: setup
!pip install emcee corner --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import emcee, corner
from scipy.integrate import quad

# Speed up tiny integrals
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Colab cell 2: load Pantheon light-curve parameters
# We'll download the official Pantheon file from GitHub.
!wget -q https://raw.githubusercontent.com/dscolnic/Pantheon/master/lcparam_full_long_zhel.txt

# Columns: name zcmb zhel dz mb dmb x1 dx1 color dcolor 3rdvar d3rdvar cov_m_s cov_m_c cov_s_c set ra dec biascor
sn = pd.read_csv("lcparam_full_long_zhel.txt",
                 delim_whitespace=True, comment="#", header=None,
                 names=["name","zcmb","zhel","dz","mb","dmb","x1","dx1",
                        "color","dcolor","3rdvar","d3rdvar",
                        "cov_m_s","cov_m_c","cov_s_c","set",
                        "ra","dec","biascor"])
print("Loaded Pantheon, N =", len(sn))

# use CMB-frame redshift 'zcmb', mag 'mb' & error 'dmb'
z = sn["zcmb"].values
mu_obs = sn["mb"].values
mu_err = sn["dmb"].values

# cut very low z to avoid uncertainties
mask = z>0.01
z = z[mask]
mu_obs = mu_obs[mask]
mu_err = mu_err[mask]
print("After z>0.01 cut, N =", len(z))

# Colab cell 3: Define cosmological quantities
c = 299792.458  # km/s
t_pl = 5.391247e-44  # s
t0 = 4.35e17         # present age ~13.8 Gyr in s
ln_ratio = np.log(t0/t_pl)

def Omega_phi_of_z(z, gamma):
    """
    Phenomenological mapping:
      phi(t)/M_pl = tanh[gamma * ln(t/t_pl)]
    approximate t(z)/t0 ~ (1+z)^(-3/2),
    so ln(t/t_pl)=ln_ratio - (3/2)*ln(1+z).
    """
    arg = gamma*(ln_ratio - 1.5*np.log1p(z))
    return np.tanh(arg)**2

def Ez(z, H0, Om, gamma):
    op = Omega_phi_of_z(z, gamma)
    return np.sqrt(Om*(1+z)**3 + (1-Om)*op)

def dL(z, H0, Om, gamma):
    # luminosity distance via simple trapezoidal rule
    zs = np.linspace(0, z, 200)
    Es = Ez(zs, H0, Om, gamma)
    integral = np.trapz(1/Es, zs)
    return (c/H0)*(1+z)*integral

def mu_theory(z, H0, Om, gamma, Moff):
    dLs = np.array([dL(zi, H0, Om, gamma) for zi in z])
    return 5*np.log10(dLs) + 25 + Moff

# Colab cell 4: log‐probability for MCMC
def log_prior(theta):
    H0, Om, gamma, Moff = theta
    if 50<H0<90 and 0.1<Om<0.5 and 0.01<gamma<0.15 and -20<Moff< -18:
        return 0.0
    return -np.inf

def log_like(theta, z, mu, muerr):
    H0, Om, gamma, Moff = theta
    mu_mod = mu_theory(z, H0, Om, gamma, Moff)
    return -0.5*np.sum(((mu-mu_mod)/muerr)**2)

def log_prob(theta, z, mu, muerr):
    lp = log_prior(theta)
    if not np.isfinite(lp): return -np.inf
    return lp + log_like(theta, z, mu, muerr)

# Colab cell 5: run MCMC
ndim, nwalkers = 4, 32
initial = np.array([75,0.3,0.085,-19.3])
pos = initial + 1e-2*np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=(z, mu_obs, mu_err))
sampler.run_mcmc(pos, 5000, progress=True)
chain = sampler.get_chain(discard=2500, flat=True)
print("Posterior samples:", chain.shape)


# Colab cell 6: corner plot & parameter estimates
labels = ["$H_0$","$\\Omega_m$","$\\gamma$","$M_{\\rm off}$"]
fig = corner.corner(chain, labels=labels, quantiles=[0.16,0.5,0.84], show_titles=True)
plt.show()

med = np.median(chain, axis=0)
err16, err84 = np.percentile(chain, [16,84], axis=0)
for lab, m, e16, e84 in zip(labels, med, err16, err84):
    print(f"{lab} = {m:.3f} (+{e84-m:.3f}, -{m-e16:.3f})")

# Colab cell 7: plot best‐fit Hubble diagram
best_H0, best_Om, best_g, best_M = med

# data
plt.errorbar(z, mu_obs, yerr=mu_err, fmt='o', ms=3, alpha=0.5, label="Pantheon SN\,Ia")
# model
zgrid = np.linspace(0.01,1.4,200)
plt.plot(zgrid, mu_theory(zgrid, best_H0, best_Om, best_g, best_M),
         'r-', lw=2, label="Best‐fit model")
plt.xlabel("Redshift $z$")
plt.ylabel("Distance modulus $\mu$")
plt.legend()
plt.title("SN Ia Hubble Diagram")
plt.show()

print("Best‐fit:\n H0 =",best_H0,"km/s/Mpc\n Omega_m =",best_Om,
      "\n gamma =",best_g,"\n M_offset =",best_M)